In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
%cd /content/drive/MyDrive/Wave-U-Net

/content/drive/MyDrive/Wave-U-Net


In [3]:
!pip install tensorflow-gpu
!pip install tqdm pystoi pesq
!pip install json5

  error: subprocess-exited-with-error
  
  × python setup.py egg_info did not run successfully.
  │ exit code: 1
  ╰─> See above for output.
  
  note: This error originates from a subprocess, and is likely not a problem with pip.
  Preparing metadata (setup.py) ... error
error: metadata-generation-failed

× Encountered error while generating package metadata.
╰─> See above for output.

note: This is an issue with the package mentioned above, not pip.
hint: See above for details.
  Preparing metadata (setup.py) ... done
  Created wheel for pesq: filename=pesq-0.0.4-cp310-cp310-linux_x86_64.whl size=262948 sha256=9733574a4863b64dc34ead9b01458e3667e40dae12b554b6764969d8ecc71af6
  Stored in directory: /root/.cache/pip/wheels/c5/4e/2c/251524370c0fdd659e99639a0fbd0ca5a782c3aafcd456b28d
Successfully built pesq


## Create Dataset .txt Files

In [ ]:
import os
import pandas as pd


In [ ]:
noisy_speech_duration = pd.read_csv('/content/drive/MyDrive/audio/noisy_speech_duration.csv')
noisy_speech_duration.head()

In [ ]:
filtered_duration = noisy_speech_duration[noisy_speech_duration['duration(s)']>1]
filtered_duration = filtered_duration[filtered_duration['duration(s)']<=3]
len(noisy_speech_duration), len(filtered_duration)

(43591, 21135)

In [ ]:
noisy_speech = filtered_duration['filename'].to_list()
clean_speech = 'common_voice_zh-TW_' + filtered_duration['filename'].str[:8] + '_filtered.mp3'
clean_speech = clean_speech.to_list()

### Train

In [ ]:
path1 = '/content/drive/MyDrive/audio/noisy_train/'
path2 = '/content/drive/MyDrive/audio/clean_train/'
files1 = os.listdir(path1)
files2 = os.listdir(path2)

filtered_files1 = [f for f in files1 if f in noisy_speech]
filtered_files2 = [f for f in files2 if f in clean_speech]

print(len(files1), len(files2))
print(len(filtered_files1), len(filtered_files2))

34537 34537
16876 16876


In [ ]:
filtered_files1.sort()
filtered_files2.sort()
print(filtered_files1[:5])
print(filtered_files2[:5])

['17367897_5-210612-A-37.mp3', '17367903_5-204114-A-29.mp3', '17367905_4-218199-F-35.mp3', '17367906_5-157204-A-16.mp3', '17367907_1-32373-A-35.mp3']
['common_voice_zh-TW_17367897_filtered.mp3', 'common_voice_zh-TW_17367903_filtered.mp3', 'common_voice_zh-TW_17367905_filtered.mp3', 'common_voice_zh-TW_17367906_filtered.mp3', 'common_voice_zh-TW_17367907_filtered.mp3']


In [ ]:
[f[:8] for f in filtered_files1] == [f.replace('common_voice_zh-TW_','').replace('_filtered.mp3','') for f in filtered_files2]

True

In [ ]:
path_write = '/content/drive/MyDrive/Wave-U-Net/Datasets/MCV_ESC/trainset.txt'
with open(path_write, 'w+') as txt:
  for i in range(len(filtered_files1)):
    string = path1 + filtered_files1[i] + ' ' + path2 + filtered_files2[i]
    #print('string', string)
    txt.write(string + '\n')

### Validation

In [ ]:
path1 = '/content/drive/MyDrive/audio/noisy_valid/'
path2 = '/content/drive/MyDrive/audio/clean_valid/'
files1 = os.listdir(path1)
files2 = os.listdir(path2)

filtered_files1 = [f for f in files1 if f in noisy_speech]
filtered_files2 = [f for f in files2 if f in clean_speech]

print(len(files1), len(files2))
print(len(filtered_files1), len(filtered_files2))

8463 8463
4021 4021


In [ ]:
filtered_files1.sort()
filtered_files2.sort()
print(filtered_files1[:5])
print(filtered_files2[:5])

['17367899_3-130998-B-28.mp3', '17367900_3-120526-B-37.mp3', '17367904_3-93010-A-18.mp3', '17368625_3-145387-A-29.mp3', '17368627_3-159347-B-36.mp3']
['common_voice_zh-TW_17367899_filtered.mp3', 'common_voice_zh-TW_17367900_filtered.mp3', 'common_voice_zh-TW_17367904_filtered.mp3', 'common_voice_zh-TW_17368625_filtered.mp3', 'common_voice_zh-TW_17368627_filtered.mp3']


In [ ]:
[f[:8] for f in filtered_files1] == [f.replace('common_voice_zh-TW_','').replace('_filtered.mp3','') for f in filtered_files2]

True

In [ ]:
path_write = '/content/drive/MyDrive/Wave-U-Net/Datasets/MCV_ESC/validset.txt'
with open(path_write, 'w+') as txt:
  for i in range(len(filtered_files1)):
    string = path1 + filtered_files1[i] + ' ' + path2 + filtered_files2[i]
    #print('string', string)
    txt.write(string + '\n')

## Train model

In [5]:
!python train.py -C config/train/1129_train_01.json

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
2024-11-29 07:19:56.936234: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 07:19:56.953851: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 07:19:56.975081: E external/

In [6]:
!python train.py -C config/train/1129_train_01.json -R

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
2024-11-29 09:12:57.623902: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 09:12:57.641571: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 09:12:57.662721: E external/

In [7]:
!python train.py -C config/train/1129_train_01.json -R

/usr/local/lib/python3.10/dist-packages/torch/utils/data/dataloader.py:617: UserWarning: This DataLoader will create 40 worker processes in total. Our suggested max number of worker in current system is 12, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  warnings.warn(
2024-11-29 10:31:12.442211: I tensorflow/core/util/port.cc:153] oneDNN custom operations are on. You may see slightly different numerical results due to floating-point round-off errors from different computation orders. To turn them off, set the environment variable `TF_ENABLE_ONEDNN_OPTS=0`.
2024-11-29 10:31:12.459682: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:485] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2024-11-29 10:31:12.481189: E external/